In [1]:
# Импортируем либы
import pandas as pd
import numpy as np
from datetime import date
import holidays
from prophet import Prophet

In [2]:
url = 'ТЗ Marketing Analyst.csv'

In [3]:
df = pd.read_csv('ТЗ Marketing Analyst.csv', parse_dates = ['Date'])


In [ ]:
Date – календарная дата
Count user id – количество пользователей с указанными параметрами
активных в соответствующий день
Count payers – количество плательщиков (накопительный итог от даты
установки)
Country – страна (двоичный код ISO)
Campaign – название компании по закупке трафика
Media source – источник трафика в котором велась закупка
Revenue – доход за соответствующий день
CPI – стоимость одной установки, которая не меняется со временем

Какая прибыль или убыток будут получены от данной закупки трафика
• На какие позитивные или негативные моменты в закупке стоит обратить
внимание? Что на Ваш взгляд можно было бы и изменить и как?
Сделайте рекомендации в отношении закупки трафика, какая компания более успешная и почему

In [4]:
df

,Date,Count user id,Count payers,Country,Campaign,Media source,CPI,Revenue
0,2020-01-02,1230,17,US,camp_1,Facebook,5.3,782.28
1,2020-01-02,983,14,CA,camp_1,Facebook,5.3,625.19
2,2020-01-02,1800,25,UK,camp_1,Facebook,5.5,1188.00
3,2020-01-02,900,1,KW,camp_1,Google,5.5,1485.00
4,2020-01-02,1100,15,AE,camp_1,Google,3.5,462.00
...,...,...,...,...,...,...,...,...
123,2020-08-02,233,53,CA,camp_3,Facebook,5.0,284.13
124,2020-08-02,174,40,UK,camp_3,Facebook,4.7,18.84
125,2020-08-02,403,110,KW,camp_3,Google,5.3,1893.58
126,2020-08-02,311,67,AE,camp_3,Google,3.8,0.10


In [48]:
df_retention = df.groupby(['Date', 'Campaign']).agg({'Count user id': 'sum'}).reset_index()

In [49]:
df_retention

,Date,Campaign,Count user id
0,2020-01-02,camp_1,8023
1,2020-01-02,camp_2,8710
2,2020-01-02,camp_3,13170
3,2020-02-02,camp_1,2427
4,2020-02-02,camp_2,2942
5,2020-02-02,camp_3,3888
6,2020-03-02,camp_1,1645
7,2020-03-02,camp_2,2357
8,2020-03-02,camp_3,2704
9,2020-04-02,camp_1,1307


In [50]:
df_retention['count_total']=np.where(df_retention['Campaign'] == 'camp_1', 8023, np.where(df_retention['Campaign'] == 'camp_2', 8710, 13170))

In [51]:
df_retention

,Date,Campaign,Count user id,count_total
0,2020-01-02,camp_1,8023,8023
1,2020-01-02,camp_2,8710,8710
2,2020-01-02,camp_3,13170,13170
3,2020-02-02,camp_1,2427,8023
4,2020-02-02,camp_2,2942,8710
5,2020-02-02,camp_3,3888,13170
6,2020-03-02,camp_1,1645,8023
7,2020-03-02,camp_2,2357,8710
8,2020-03-02,camp_3,2704,13170
9,2020-04-02,camp_1,1307,8023


In [52]:
df_retention['retention'] = df_retention['Count user id']/df_retention['count_total']*100
df_retention

,Date,Campaign,Count user id,count_total,retention
0,2020-01-02,camp_1,8023,8023,100.000000
1,2020-01-02,camp_2,8710,8710,100.000000
2,2020-01-02,camp_3,13170,13170,100.000000
3,2020-02-02,camp_1,2427,8023,30.250530
4,2020-02-02,camp_2,2942,8710,33.777268
5,2020-02-02,camp_3,3888,13170,29.521640
6,2020-03-02,camp_1,1645,8023,20.503552
7,2020-03-02,camp_2,2357,8710,27.060850
8,2020-03-02,camp_3,2704,13170,20.531511
9,2020-04-02,camp_1,1307,8023,16.290664


In [55]:
retention_pivot = df_retention.pivot_table(index='Campaign', columns='Date',
                                      values='retention', aggfunc='sum')
retention_pivot.round(3).fillna('')

Date,2020-01-02,2020-02-02,2020-03-02,2020-04-02,2020-05-02,2020-06-02,2020-07-02,2020-08-02
Campaign,,,,,,,,
camp_1,100.0,30.251,20.504,16.291,14.284,13.175,11.828,10.931
camp_2,100.0,33.777,27.061,22.503,21.240,20.092,19.369,18.370
camp_3,100.0,29.522,20.532,17.092,15.057,12.893,12.916,12.065


In [5]:
df_1 = df.query('Date == "2020-01-02"')

In [6]:
df_1['total_cost'] = df_1['Count user id']*df_1['CPI']

C:\Users\st\AppData\Local\Temp\ipykernel_2960\3675859173.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['total_cost'] = df_1['Count user id']*df_1['CPI']


In [11]:
df_1 = df_1.groupby('Campaign').agg({'total_cost': 'sum'}).reset_index()

In [12]:
df_1

,Campaign,total_cost
0,camp_1,32840.9
1,camp_2,20537.0
2,camp_3,57556.0


In [13]:
df_2 = df.groupby(['Campaign']).agg({'Revenue': 'sum'}).reset_index()

In [14]:
df_total = df_2.merge(df_1, on = 'Campaign')

In [15]:
df_total

,Campaign,Revenue,total_cost
0,camp_1,14402.97,32840.9
1,camp_2,7577.93,20537.0
2,camp_3,28858.60,57556.0


In [18]:
df_total['share'] = round(df_total['Revenue']/df_total['total_cost']*100, 2)

In [19]:
df_total

,Campaign,Revenue,total_cost,share
0,camp_1,14402.97,32840.9,43.86
1,camp_2,7577.93,20537.0,36.90
2,camp_3,28858.60,57556.0,50.14


In [ ]:
#

In [63]:
df_first = df.query('Date == "2020-01-02"')
df_first

,Date,Count user id,Count payers,Country,Campaign,Media source,CPI,Revenue
0,2020-01-02,1230,17,US,camp_1,Facebook,5.30,782.28
1,2020-01-02,983,14,CA,camp_1,Facebook,5.30,625.19
2,2020-01-02,1800,25,UK,camp_1,Facebook,5.50,1188.00
3,2020-01-02,900,1,KW,camp_1,Google,5.50,1485.00
4,2020-01-02,1100,15,AE,camp_1,Google,3.50,462.00
5,2020-01-02,2010,28,EG,camp_1,Google,1.20,265.32
6,2020-01-02,2820,23,RU,camp_2,Google,0.80,248.16
7,2020-01-02,1320,11,BR,camp_2,Google,0.85,123.42
8,2020-01-02,2620,13,CN,camp_2,Facebook,3.20,855.17
9,2020-01-02,1950,23,TW,camp_2,Facebook,4.50,1140.75


In [64]:
df_first= df_first[['Campaign', 'Country', 'Media source', 'Count user id']]

In [65]:
df_first = df_first.groupby(['Campaign', 'Country', 'Media source']).agg({'Count user id': 'sum'}).reset_index()
df_first

,Campaign,Country,Media source,Count user id
0,camp_1,AE,Google,1100
1,camp_1,CA,Facebook,983
2,camp_1,EG,Google,2010
3,camp_1,KW,Google,900
4,camp_1,UK,Facebook,1800
5,camp_1,US,Facebook,1230
6,camp_2,BR,Google,1320
7,camp_2,CN,Facebook,2620
8,camp_2,RU,Google,2820
9,camp_2,TW,Facebook,1950


In [70]:
df_country_media = df.groupby(['Campaign',  'Country', 'Media source']).agg({'Revenue': 'sum'}).reset_index()

In [71]:
df_country_media

,Campaign,Country,Media source,Revenue
0,camp_1,AE,Google,1732.50
1,camp_1,CA,Facebook,1979.76
2,camp_1,EG,Google,988.92
3,camp_1,KW,Google,2970.00
4,camp_1,UK,Facebook,4059.00
5,camp_1,US,Facebook,2672.79
6,camp_2,BR,Google,504.89
7,camp_2,CN,Facebook,2246.93
8,camp_2,RU,Google,789.60
9,camp_2,TW,Facebook,4036.51


In [73]:
df_by_country_total = df_country_media.merge(df_first, on =['Campaign', 'Country', 'Media source'])
df_by_country_total

,Campaign,Country,Media source,Revenue,Count user id
0,camp_1,AE,Google,1732.50,1100
1,camp_1,CA,Facebook,1979.76,983
2,camp_1,EG,Google,988.92,2010
3,camp_1,KW,Google,2970.00,900
4,camp_1,UK,Facebook,4059.00,1800
5,camp_1,US,Facebook,2672.79,1230
6,camp_2,BR,Google,504.89,1320
7,camp_2,CN,Facebook,2246.93,2620
8,camp_2,RU,Google,789.60,2820
9,camp_2,TW,Facebook,4036.51,1950


In [74]:
#ARPU за 8 дней
df_by_country_total['ARPU'] = df_by_country_total['Revenue']/df_by_country_total['Count user id']

In [75]:
df_by_country_total

,Campaign,Country,Media source,Revenue,Count user id,ARPU
0,camp_1,AE,Google,1732.50,1100,1.575000
1,camp_1,CA,Facebook,1979.76,983,2.013998
2,camp_1,EG,Google,988.92,2010,0.492000
3,camp_1,KW,Google,2970.00,900,3.300000
4,camp_1,UK,Facebook,4059.00,1800,2.255000
5,camp_1,US,Facebook,2672.79,1230,2.173000
6,camp_2,BR,Google,504.89,1320,0.382492
7,camp_2,CN,Facebook,2246.93,2620,0.857607
8,camp_2,RU,Google,789.60,2820,0.280000
9,camp_2,TW,Facebook,4036.51,1950,2.070005


In [32]:
df_by_country_total.groupby('Campaign')['Count user id'].sum()

Campaign
camp_1     8023
camp_2     8710
camp_3    13170
Name: Count user id, dtype: int64

In [38]:
df_first

,Date,Count user id,Count payers,Country,Campaign,Media source,CPI,Revenue
0,2020-01-02,1230,17,US,camp_1,Facebook,5.30,782.28
1,2020-01-02,983,14,CA,camp_1,Facebook,5.30,625.19
2,2020-01-02,1800,25,UK,camp_1,Facebook,5.50,1188.00
3,2020-01-02,900,1,KW,camp_1,Google,5.50,1485.00
4,2020-01-02,1100,15,AE,camp_1,Google,3.50,462.00
5,2020-01-02,2010,28,EG,camp_1,Google,1.20,265.32
6,2020-01-02,2820,23,RU,camp_2,Google,0.80,248.16
7,2020-01-02,1320,11,BR,camp_2,Google,0.85,123.42
8,2020-01-02,2620,13,CN,camp_2,Facebook,3.20,855.17
9,2020-01-02,1950,23,TW,camp_2,Facebook,4.50,1140.75


In [77]:
df_first = df.query('Date == "2020-01-02"')

In [79]:
df_first_cpi = df_first[['Country', 'Campaign', 'Media source', 'CPI']]
df_first_cpi

,Country,Campaign,Media source,CPI
0,US,camp_1,Facebook,5.30
1,CA,camp_1,Facebook,5.30
2,UK,camp_1,Facebook,5.50
3,KW,camp_1,Google,5.50
4,AE,camp_1,Google,3.50
5,EG,camp_1,Google,1.20
6,RU,camp_2,Google,0.80
7,BR,camp_2,Google,0.85
8,CN,camp_2,Facebook,3.20
9,TW,camp_2,Facebook,4.50


In [80]:
df_by_country_total = df_by_country_total.merge(df_first_cpi, on = ['Country', 'Campaign', 'Media source'])
df_by_country_total

,Campaign,Country,Media source,Revenue,Count user id,ARPU,CPI
0,camp_1,AE,Google,1732.50,1100,1.575000,3.50
1,camp_1,CA,Facebook,1979.76,983,2.013998,5.30
2,camp_1,EG,Google,988.92,2010,0.492000,1.20
3,camp_1,KW,Google,2970.00,900,3.300000,5.50
4,camp_1,UK,Facebook,4059.00,1800,2.255000,5.50
5,camp_1,US,Facebook,2672.79,1230,2.173000,5.30
6,camp_2,BR,Google,504.89,1320,0.382492,0.85
7,camp_2,CN,Facebook,2246.93,2620,0.857607,3.20
8,camp_2,RU,Google,789.60,2820,0.280000,0.80
9,camp_2,TW,Facebook,4036.51,1950,2.070005,4.50


In [81]:
df_by_country_total['diff'] = round(df_by_country_total.ARPU/df_by_country_total.CPI*100, 1)

In [82]:
df_by_country_total

,Campaign,Country,Media source,Revenue,Count user id,ARPU,CPI,diff
0,camp_1,AE,Google,1732.50,1100,1.575000,3.50,45.0
1,camp_1,CA,Facebook,1979.76,983,2.013998,5.30,38.0
2,camp_1,EG,Google,988.92,2010,0.492000,1.20,41.0
3,camp_1,KW,Google,2970.00,900,3.300000,5.50,60.0
4,camp_1,UK,Facebook,4059.00,1800,2.255000,5.50,41.0
5,camp_1,US,Facebook,2672.79,1230,2.173000,5.30,41.0
6,camp_2,BR,Google,504.89,1320,0.382492,0.85,45.0
7,camp_2,CN,Facebook,2246.93,2620,0.857607,3.20,26.8
8,camp_2,RU,Google,789.60,2820,0.280000,0.80,35.0
9,camp_2,TW,Facebook,4036.51,1950,2.070005,4.50,46.0


In [43]:
df_total

,Campaign,Revenue,total_cost,share
0,camp_1,14402.97,32840.9,43.86
1,camp_2,7577.93,20537.0,36.90
2,camp_3,28858.60,57556.0,50.14


In [44]:
df_by_country_total.groupby('Campaign')['Count user id'].sum()

Campaign
camp_1     8023
camp_2     8710
camp_3    13170
Name: Count user id, dtype: int64

In [56]:
retention_pivot

Date,2020-01-02,2020-02-02,2020-03-02,2020-04-02,2020-05-02,2020-06-02,2020-07-02,2020-08-02
Campaign,,,,,,,,
camp_1,100.0,30.250530,20.503552,16.290664,14.283934,13.174623,11.828493,10.931073
camp_2,100.0,33.777268,27.060850,22.502870,21.239954,20.091848,19.368542,18.369690
camp_3,100.0,29.521640,20.531511,17.091875,15.056948,12.892938,12.915718,12.065300
